In [42]:
#intent classification is process of classifying customer's intent by analyzing the language they use
#it is the central aspect of chatbot conversation
#2 phases:training phrase and preset response
#methods for intent classification:
#BoW
#TF-IDF
#CNN with Glove
#BERT

In [43]:
import os
from apikey import apikey
os.environ['HUGGINGFACEHUB_API_TOKEN']=apikey

In [44]:
#!pip install datasets
from datasets import load_dataset

In [45]:
dataset=load_dataset("banking77",split="train")

C:\Users\user\.cache\huggingface\modules\datasets_modules\datasets\banking77\9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4\banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(


In [46]:
dataset[0]

{'text': 'I am still waiting on my card?', 'label': 11}

In [47]:
len(dataset)

10003

In [48]:
import pandas as pd
df=pd.DataFrame(dataset)
df

,text,label
0,I am still waiting on my card?,11
1,What can I do if my card still hasn't arrived ...,11
2,I have been waiting over a week. Is the card s...,11
3,Can I track my card while it is in the process...,11
4,"How do I know if I will get my card, or if it ...",11
...,...,...
9998,You provide support in what countries?,24
9999,What countries are you supporting?,24
10000,What countries are getting support?,24
10001,Are cards available in the EU?,24


In [49]:
from transformers import BertTokenizer,BertForSequenceClassification
import torch

In [50]:
labels=df['label'].unique()
labels

array([11, 13, 32, 17, 34, 46, 36, 12,  4, 14, 33, 41,  1, 49, 23, 56, 47,
        8, 60, 75, 15, 66, 54, 40, 10, 61,  6, 16, 30, 74, 68, 38, 73, 62,
       29, 22,  3, 28, 44, 26, 45, 42, 52, 27, 51, 25, 48, 55, 18, 63, 70,
       67, 53, 21,  7, 64, 50, 35, 65, 71, 39, 58, 43, 72, 76, 37, 59,  5,
       20, 31, 57,  0, 19,  9,  2, 69, 24], dtype=int64)

In [51]:
len(labels)

77

In [52]:
def classifier(text):
    # Load pre-trained BERT model and tokenizer
    model_name = "bert-large-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(labels))

    # Tokenize and encode the input
    inputs = tokenizer.encode_plus(text, add_special_tokens=True,padding=True, return_tensors="pt")
    print(inputs)
    # Get predictions
    logits = model(**inputs).logits
    print(logits)
    # Get predicted label 
    predicted_label = labels[torch.argmax(logits).item()]

    return predicted_label

In [53]:
df['text'][0]

'I am still waiting on my card?'

In [54]:
while True:
    text = input()
    if text.lower() == "exit":
            break
# Classify the sentiment
    output = classifier(text)
    print("Predicted output:",output)


I am still waiting on my card


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[ 101, 1045, 2572, 2145, 3403, 2006, 2026, 4003,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[-0.8534, -0.1235,  0.0649,  0.1894, -0.5885,  0.3073,  1.0742, -0.1464,
         -0.4807, -0.9743,  0.8795,  0.1553, -0.2422, -0.5565,  0.2599, -0.3448,
          1.0411,  0.4346, -0.2175, -0.9908, -1.1609, -0.0093,  0.6316,  0.5810,
          0.4748, -0.3263,  0.3702, -0.3331, -0.2253,  0.3205, -0.2322, -0.1730,
         -0.3428, -0.6191,  0.4870,  0.2795,  0.5951,  0.0585,  0.1415,  0.0868,
         -0.3747, -0.7496,  0.2090, -0.6517,  0.5493,  0.1231,  1.0092, -0.0756,
          0.3626, -0.0907,  0.0123,  0.2949, -0.1673, -0.2499, -0.4292,  0.5598,
          0.3320, -0.1618, -0.2829, -0.0526,  0.7710,  0.6947,  0.6272, -0.9754,
         -0.6762,  0.8161,  0.4322, -0.3198, -0.2883,  0.2255, -0.6115,  0.1498,
         -1.1203, -0.2721,  0.2935,  0.5227,  0.2310]],
       grad_fn=<Addmm

In [55]:
import os
from apikey import apikey
os.environ['HUGGINGFACEHUB_API_TOKEN']=apikey

In [56]:
from langchain import HuggingFaceHub,LLMChain
from langchain.prompts import PromptTemplate

In [58]:
from transformers import pipeline
pipe = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\user\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [60]:
hub_llm=HuggingFaceHub(repo_id="philschmid/BERT-Banking77")

ValidationError: 1 validation error for HuggingFaceHub
__root__
  Got invalid task text-classification, currently only ('text2text-generation', 'text-generation', 'summarization') are supported (type=value_error)

In [ ]:
prompt=PromptTemplate(
        input_variables=['output'],
        template="The intent inside the message is {output} ",
        model_kwargs={'temperature':0.1,'max_length':100}
)

In [ ]:
hub_chain=LLMChain(prompt=prompt,llm=hub_llm,verbose=True)

In [ ]:
print(hub_chain.run(text))